In [2]:
#import modin.pandas as pd
from pandas_profiling import ProfileReport
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import (GridSearchCV, StratifiedKFold,
                                     train_test_split)
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, mean_absolute_error,
                             roc_auc_score)
from sklearn.impute import SimpleImputer
from sklearn.ensemble import (AdaBoostClassifier, RandomForestClassifier,
                              RandomForestRegressor)
import xgboost as xgb
import scipy.stats as stats
import requests
import pyforest
import cv2 as cv
import bs4 as bs
import warnings
import time
import re
import random
import os
import math
import datetime
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline
# from autoviz.AutoViz_Class import AutoViz_Class

%config InlineBackend.figure_format = 'retina'

#import keras as ks
# import tensorflow as tf

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
sns.set(style='whitegrid', color_codes=True, font_scale=1.5,
        palette='plasma', font='Arial', rc={'figure.figsize': (8, 6)})


In [4]:
df=sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.00,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.00,1,0,71.28,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.00,0,0,7.92,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.00,1,0,53.10,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.00,0,0,8.05,S,Third,man,True,NaN,Southampton,no,True


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [6]:
df.shape

(891, 15)

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
survived,891.00,0.38,0.49,0.00,0.00,0.00,1.00,1.00
pclass,891.00,2.31,0.84,1.00,2.00,3.00,3.00,3.00
age,714.00,29.70,14.53,0.42,20.12,28.00,38.00,80.00
sibsp,891.00,0.52,1.10,0.00,0.00,0.00,1.00,8.00
parch,891.00,0.38,0.81,0.00,0.00,0.00,0.00,6.00
fare,891.00,32.20,49.69,0.00,7.91,14.45,31.00,512.33


We will be predicting the survival of the passenger based on given data

In [67]:
y=df['survived']
X=df.drop(['survived','class','deck'],axis=1)
X= X.select_dtypes(exclude=['object'])

In [68]:
y

1      1
3      1
6      0
10     1
11     1
      ..
871    1
872    0
879    1
887    1
889    1
Name: survived, Length: 182, dtype: int64

In [69]:
X

,pclass,age,sibsp,parch,fare,adult_male,alone
1,1,38.00,1,0,71.28,False,False
3,1,35.00,1,0,53.10,False,False
6,1,54.00,0,0,51.86,True,True
10,3,4.00,1,1,16.70,False,False
11,1,58.00,0,0,26.55,False,True
...,...,...,...,...,...,...,...
871,1,47.00,1,1,52.55,False,False
872,1,33.00,0,0,5.00,True,True
879,1,56.00,0,1,83.16,False,False
887,1,19.00,0,0,30.00,False,True


CASE 1 - Remove all missing rows

In [70]:
X.isna().sum()

pclass        0
age           0
sibsp         0
parch         0
fare          0
adult_male    0
alone         0
dtype: int64

In [71]:
X.dropna(inplace=True)

In [72]:
X.isna().sum()

pclass        0
age           0
sibsp         0
parch         0
fare          0
adult_male    0
alone         0
dtype: int64

In [73]:
X.shape

(182, 7)

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [75]:
X_train

,pclass,age,sibsp,parch,fare,adult_male,alone
331,1,45.50,0,0,28.50,True,True
336,1,29.00,1,0,66.60,True,False
193,2,3.00,1,1,26.00,False,False
75,3,25.00,0,0,7.65,True,True
248,1,37.00,1,1,52.55,True,False
...,...,...,...,...,...,...,...
523,1,44.00,0,1,57.98,False,False
92,1,46.00,1,0,61.17,True,False
462,1,47.00,0,0,38.50,True,True
879,1,56.00,0,1,83.16,False,False


In [76]:
# Random Forest Classifier - whether person survived or not
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
pred = model.predict(X_test)
accuracy = accuracy_score(y_test, pred)

RandomForestClassifier(random_state=42)

In [78]:
pred
len(pred)

array([0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1], dtype=int64)

37

In [79]:
accuracy

0.7567567567567568

In [80]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.64      0.69      0.67        13
           1       0.83      0.79      0.81        24

    accuracy                           0.76        37
   macro avg       0.73      0.74      0.74        37
weighted avg       0.76      0.76      0.76        37



CASE 2 - Using SimpleImputer() sklearn library

In [81]:
df=sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.00,1,0,7.25,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.00,1,0,71.28,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.00,0,0,7.92,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.00,1,0,53.10,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.00,0,0,8.05,S,Third,man,True,NaN,Southampton,no,True


In [85]:
df.isna().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [89]:
# Impute missing values
imputer = SimpleImputer(strategy='median')
imputed_X_train = imputer.fit_transform(X_train)
imputed_X_test = imputer.transform(X_test)
imp_X_train = pd.DataFrame(imputed_X_train, columns=X_train.columns)
imp_X_test = pd.DataFrame(imputed_X_test, columns=X_test.columns)

In [92]:
imp_X_train

,pclass,age,sibsp,parch,fare,adult_male,alone
0,1.00,45.50,0.00,0.00,28.50,1.00,1.00
1,1.00,29.00,1.00,0.00,66.60,1.00,0.00
2,2.00,3.00,1.00,1.00,26.00,0.00,0.00
3,3.00,25.00,0.00,0.00,7.65,1.00,1.00
4,1.00,37.00,1.00,1.00,52.55,1.00,0.00
...,...,...,...,...,...,...,...
140,1.00,44.00,0.00,1.00,57.98,0.00,0.00
141,1.00,46.00,1.00,0.00,61.17,1.00,0.00
142,1.00,47.00,0.00,0.00,38.50,1.00,1.00
143,1.00,56.00,0.00,1.00,83.16,0.00,0.00


In [93]:
# Training model 
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(imp_X_train, y_train)
pred = model.predict(imp_X_test)
accuracy = accuracy_score(y_test, pred)
pred
accuracy

RandomForestClassifier(random_state=42)

array([0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1], dtype=int64)

0.7567567567567568

In [96]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.64      0.69      0.67        13
           1       0.83      0.79      0.81        24

    accuracy                           0.76        37
   macro avg       0.73      0.74      0.74        37
weighted avg       0.76      0.76      0.76        37

